<a href="https://colab.research.google.com/github/malowana/NLP-Disaster-Tweets-with-Flask/blob/main/NLP_disasterTweetsv2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install scikit-plot
!pip install neptune-client
!pip install opendatasets --upgrade

In [ ]:
!pip install texthero

In [ ]:
!pip3 install catboost
!pip install scikit-plot

In [ ]:
import pandas as pd
import numpy as np
import json
import neptune
import neptune.new as neptune
import opendatasets as od
import texthero as hero
import pickle

from gensim.utils import simple_preprocess
import catboost as ctb
import xgboost as xgb
from gensim.models import FastText
from scikitplot.estimators import plot_learning_curve
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold, cross_val_score
from gensim.models import Word2Vec, Phrases

## **Load data and initial check**

In [ ]:
# Kaggle authorization data: {'username': '...', 'key': '...'}
with open('/content/drive/MyDrive/Konkursy Kaggle/Disaster Tweets/kaggle.json') as f:
   kaggle_authorization = json.load(f)

# Neptune authorization data: {'project': '...', 'api_token': '...'}
with open('/content/drive/MyDrive/Konkursy Kaggle/Disaster Tweets/neptune-disastertweets.json') as f:
   neptune_authorization = json.load(f)

In [ ]:
# Token should be set in the system settings or directly below
#PROJECT_NAME = 'DataWorkshop-Foundation/Predicting-cancellation-of-visit' 
PROJECT_NAME = neptune_authorization['project'] 
MY_TOKEN = neptune_authorization['api_token']
project = neptune.init(project = PROJECT_NAME, api_token=MY_TOKEN)
project

https://app.neptune.ai/paulina21/Kaggle-Disaster-Tweets/e/KAG-25
Remember to stop your run once you’ve finished logging your metadata (https://docs.neptune.ai/api-reference/run#stop). It will be stopped automatically only when the notebook kernel/interactive console is terminated.


In [ ]:
kaggle_url = "https://www.kaggle.com/c/nlp-getting-started/data"
filename_test = r"/content/username/nlp-getting-started/test.csv"
filename_train = r"/content/username/nlp-getting-started/train.csv"
path = r"."  # for saving data
author = "Paulina Sz"  # for Neptune

In [ ]:
# Upload directly from Kaggle:
od.download(kaggle_url, *kaggle_authorization)

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: paulinasz
Your Kaggle Key: ··········


100%|██████████| 593k/593k [00:00<00:00, 51.8MB/s]


Extracting archive username/nlp-getting-started/nlp-getting-started.zip to username/nlp-getting-started


In [ ]:
df_train = pd.read_csv(filename_train)
df_test = pd.read_csv(filename_test)
print("Shape:", df_train.shape)
print("Duplicated:", df_train.duplicated().sum())
df_train.head()

Shape: (7613, 5)
Duplicated: 0


,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [ ]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        7613 non-null   int64 
 1   keyword   7552 non-null   object
 2   location  5080 non-null   object
 3   text      7613 non-null   object
 4   target    7613 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 297.5+ KB


In [ ]:
# Check target (ratio)
df_train["target"].value_counts(normalize=True, dropna=False)

0    0.57034
1    0.42966
Name: target, dtype: float64

In [ ]:
#check if missing values
def is_missing(train):
  for column in train.columns:
      missing = column, train[column].isnull().sum()
      if missing[1] == 0: continue
      print(missing)

is_missing(df_train)
is_missing(df_test)

('keyword', 61)
('location', 2533)
('keyword', 26)
('location', 1105)


In [ ]:
#concatenate the subsets
all = pd.concat([df_train, df_test], sort=False)

## **Feature engineering**

In [ ]:
def get_all_hastags(list):
    for word in list:
      if word.startswith('#'):
        return word

In [ ]:
#fill missing keywords from # from text
all["text_list"] = all["text"].map(lambda x: x.split())
all["text_hashtags"] = all["text_list"].map(get_all_hastags)
all['keyword'].fillna(all["text_hashtags"], inplace=True)
if 'text_hashtags' in all: del all['text_hashtags']
all.head()

,id,keyword,location,text,target,text_list
0,1,#earthquake,NaN,Our Deeds are the Reason of this #earthquake M...,1.0,"[Our, Deeds, are, the, Reason, of, this, #eart..."
1,4,None,NaN,Forest fire near La Ronge Sask. Canada,1.0,"[Forest, fire, near, La, Ronge, Sask., Canada]"
2,5,None,NaN,All residents asked to 'shelter in place' are ...,1.0,"[All, residents, asked, to, 'shelter, in, plac..."
3,6,#wildfires,NaN,"13,000 people receive #wildfires evacuation or...",1.0,"[13,000, people, receive, #wildfires, evacuati..."
4,7,#Alaska,NaN,Just got sent this photo from Ruby #Alaska as ...,1.0,"[Just, got, sent, this, photo, from, Ruby, #Al..."


In [ ]:
all.location = all.location.fillna("other")

In [ ]:
#location
all.loc[all['location'].str.contains('Iraq|Pakistan'), 'location'] = 'Asia'
all.loc[all['location'].str.contains('Las Vegas'), 'location'] = 'Las Vegas'
all.loc[all['location'].str.contains('Irleand|Dublin'), 'location'] = 'Irleand'
all.loc[all['location'].str.contains('OH|Ohio'), 'location'] = 'Ohio'
all.loc[all['location'].str.contains('Brazil'), 'location'] = 'Brasil'
all.loc[all['location'].str.contains('MO|Kansas City'), 'location'] = 'Missouri'
all.loc[all['location'].str.contains('umbai|India'), 'location'] = 'India'
all.loc[all['location'].str.contains('Italy|France|Switzerland|Netherlands|Germany|Sweden|Geneva|Rome|Spain'), 'location'] = 'Europe'
all.loc[all['location'].str.contains('Nigeria|Lagos'), 'location'] = 'Nigeria'
all.loc[all['location'].str.contains('Japan|Tokyo'), 'location'] = 'Japan'
all.loc[all['location'].str.contains('Virginia|VA'), 'location'] = 'Virginia'
all.loc[all['location'].str.contains('Hong Kong|china|China'), 'location'] = 'China'
all.loc[all['location'].str.contains('Edinburgh'), 'location'] = 'Scotland'
all.loc[all['location'].str.contains('NC|North Carolina'), 'location'] = 'North Carolina'
all.loc[all['location'].str.contains('Portland'), 'location'] = 'Portland'
all.loc[all['location'].str.contains('Boston|WI|PA|IA|Poenix|Georgia|America|United States|Unites States|U.S.A|US|U.S.|Wisconsin|New Hampshire|Arizona|Midwest|Indiana|Maryland|Pennsylvania|Puerto Rico|Oregon|Michigan'), 'location'] = 'USA'
all.loc[all['location'].str.contains('NJ|NY|New York|NYC|york|nyc'), 'location'] = 'New York'
all.loc[all['location'].str.contains('Sydney'), 'location'] = 'Sydney'
all.loc[all['location'].str.contains('ondon|United Kingdom|England|Coventry|Birmingham|Paignton|Liverpool|Glasgow|Newcastle|UK|Manchester'), 'location'] = 'UK'
all.loc[all['location'].str.contains('Australia|Melbourne|Sydney'), 'location'] = 'Australia'
all.loc[all['location'].str.contains('California|San Diego|Sacramento|San Francisco'), 'location'] = 'California'
all.loc[all['location'].str.contains('Washington'), 'location'] = 'Washington'
all.loc[all['location'].str.contains('Chicago'), 'location'] = 'Chicago'
all.loc[all['location'].str.contains('Los Angeles|Subconscious|LA'), 'location'] = 'Los Angeles'
all.loc[all['location'].str.contains('Seattle, WA'), 'location'] = 'Seattle'
all.loc[all['location'].str.contains('World|orldwide|Earth|Global|The Universe'), 'location'] = 'Everywhere'
all.loc[all['location'].str.contains('TX|Texas|Houston'), 'location'] = 'Texas'
all.loc[all['location'].str.contains('CA|Canada|Toronto|Vancouver|canada|Calgary'), 'location'] = 'Canada'
all.loc[all['location'].str.contains('TN'), 'location'] = 'Tennessee'
all.loc[all['location'].str.contains('FL|Orlando'), 'location'] = 'Florida'
all.loc[all['location'].str.contains('304|house|hell|Taylor|Tunes|stairs|Coffee|ss|Money|Happily Married|MAD as Hell|Billionaires|Word|Pedophile|Narnia|Your screen|Here|Jupiter|Anonymous|really|Breaking|where|All|Some|Innerhalb|Sky|BROKE|Can|Sky'), 'location'] = 'other'
all.loc[all['location'].str.contains('Lagos'), 'location'] = 'Lagos'
all.loc[all['location'].str.contains('Denver'), 'location'] = 'Denver'
all.loc[all['location'].str.contains('New Jersey'), 'location'] = 'New Jersey'
all.loc[all['location'].str.contains('Calgary'), 'location'] = 'Calgary'
all.loc[all['location'].str.contains('Atlanta|atlanta'), 'location'] = 'Atlanta'
all.loc[all['location'].str.contains('Detroit'), 'location'] = 'Michigan'

In [ ]:
def new_features(all):
  all['length_text'] = all['text'].str.len()
  all['length_keyword'] = all['keyword'].str.len()
  all['length_location'] = all['location'].str.len()
  all['word_count'] = all['text'].str.split().str.len()
  all['is_upper_text'] = all['text'].str.count(r'[A-Z]')
  all['is_digit_text'] = all['text'].str.count(r'[0-9]')
  all['%_upper_text'] = all['is_upper_text']/all['length_text']
  all['is_#'] = all['text'].str.count(r'#')
  all.loc[(all['text'].str.contains("http")), 'is_url'] = 1
  all.loc[all['is_url'].isnull(), 'is_url'] = 0
  all['is_@'] = all['text'].str.count(r'@')
  all['is_?'] = all['text'].str.count(r'\?')
  all['is_!'] = all['text'].str.count(r'\!')

In [ ]:
new_features(all)

In [ ]:
import re
def re_urls(text, replace_for="URL"):
    return re.sub(r'https?://\S+', replace_for, text) 

def re_digits(text, replace_for=' '): 
    result = re.sub(r'\d+', replace_for, text) 
    return result

def re_hashtags(text, replace_for=' '): 
    result = re.sub(r'#', replace_for, text) 
    return result

def preprocessing(doc):
    doc = re_urls(doc)
    doc = re_digits(doc)
    doc = re_hashtags(doc)
    return doc

all['text_after_preprocessing'] = all['text'].map(preprocessing).values

In [ ]:
all[all.location=="other"].sample(5)

,id,keyword,location,text,target,text_list,length_text,length_keyword,length_location,word_count,is_upper_text,is_digit_text,%_upper_text,is_#,is_url,is_@,is_?,is_!,text_after_preprocessing
6951,9973,tsunami,other,#sing #tsunami Beginners #computer tutorial.: ...,0.0,"[#sing, #tsunami, Beginners, #computer, tutori...",133,7.0,5,16,20,1,0.150376,3,1.0,0,0,0,sing tsunami Beginners computer tutorial.: ...
5103,7280,nuclear%20disaster,other,Any disaster impairs mental health especially ...,1.0,"[Any, disaster, impairs, mental, health, espec...",97,18.0,5,10,5,0,0.051546,0,1.0,0,0,0,Any disaster impairs mental health especially ...
5421,7737,panicking,other,When he lets you drive his truck and you start...,0.0,"[When, he, lets, you, drive, his, truck, and, ...",124,9.0,5,20,5,3,0.040323,0,1.0,0,6,0,When he lets you drive his truck and you start...
5958,8509,screaming,other,/ it's fine baby I was screaming at the TV x h...,0.0,"[/, it's, fine, baby, I, was, screaming, at, t...",68,9.0,5,12,10,1,0.147059,0,1.0,0,0,0,/ it's fine baby I was screaming at the TV x URL
3348,4793,evacuated,other,Hotel evacuated after fire on Shore Drive in N...,1.0,"[Hotel, evacuated, after, fire, on, Shore, Dri...",75,9.0,5,10,6,2,0.080000,0,1.0,0,0,0,Hotel evacuated after fire on Shore Drive in N...


In [ ]:
all["text_hero"] = hero.clean(all["text_after_preprocessing"])
all.head()

,id,keyword,location,text,target,text_list,length_text,length_keyword,length_location,word_count,is_upper_text,is_digit_text,%_upper_text,is_#,is_url,is_@,is_?,is_!,text_after_preprocessing,text_hero
0,1,#earthquake,other,Our Deeds are the Reason of this #earthquake M...,1.0,"[Our, Deeds, are, the, Reason, of, this, #eart...",69,11.0,5,13,10,0,0.144928,1,0.0,0,0,0,Our Deeds are the Reason of this earthquake M...,deeds reason earthquake may allah forgive us
1,4,None,other,Forest fire near La Ronge Sask. Canada,1.0,"[Forest, fire, near, La, Ronge, Sask., Canada]",38,NaN,5,7,5,0,0.131579,0,0.0,0,0,0,Forest fire near La Ronge Sask. Canada,forest fire near la ronge sask canada
2,5,None,other,All residents asked to 'shelter in place' are ...,1.0,"[All, residents, asked, to, 'shelter, in, plac...",133,NaN,5,22,2,0,0.015038,0,0.0,0,0,0,All residents asked to 'shelter in place' are ...,residents asked shelter place notified officer...
3,6,#wildfires,other,"13,000 people receive #wildfires evacuation or...",1.0,"[13,000, people, receive, #wildfires, evacuati...",65,10.0,5,8,1,5,0.015385,1,0.0,0,0,0,", people receive wildfires evacuation order...",people receive wildfires evacuation orders cal...
4,7,#Alaska,other,Just got sent this photo from Ruby #Alaska as ...,1.0,"[Just, got, sent, this, photo, from, Ruby, #Al...",88,7.0,5,16,3,0,0.034091,2,0.0,0,0,0,Just got sent this photo from Ruby Alaska as ...,got sent photo ruby alaska smoke wildfires pou...


In [ ]:
#all_1 = all[all['target']==0]
#all_0 = all[all['target']==1]

In [ ]:
#hero.visualization.wordcloud(all_0["text_hero"])

In [ ]:
#hero.visualization.wordcloud(all_1["text_hero"])

## **Create models**

In [ ]:
def get_feats(df, black_list=['target', 'id' ]):
  num_feats = df.select_dtypes(np.number).columns
  return [x for x in num_feats if x not in black_list]

#vectorization & vectors average
def get_doc2vec_X(model, tokens):
    def __calc_doc2vec(words):   #vectorization
        return np.mean([model.wv[w] for w in words if w in model.wv], axis=0)

    X = tokens.map(__calc_doc2vec)
    default_vector = X[ False == X.isnull() ].mean()
    return np.stack( X.map(lambda x: default_vector if str(x) == 'nan' else x) )

def draw_feature_importances(model, features):
    importances = model.feature_importances_
    indices = np.argsort(importances)[::-1]

    plt.figure(figsize=(10, 5))
    plt.title("Feature importances")
    plt.bar(range(X.shape[1]), model.feature_importances_[indices],
           color="b", align="center")
    plt.xticks(range(X.shape[1]), [ features[x] for x in indices])
    plt.xticks(rotation=90)
    plt.xlim([-1, X.shape[1]])
    plt.show()

def log_to_neptune(model, params, feats, tags,metric):
  project = neptune.init(project = PROJECT_NAME, api_token=MY_TOKEN)
  project["model"] = params
  project["parameters"] = params
  project["feats"] = feats
  project["sys/tags"].add(tags)
  project["train/accuracy"] = metric
  project.stop()

In [ ]:
#split to test/train again
train2 = all[ ~all['target'].isnull() ].copy()
test2 = all[ all['target'].isnull() ].copy()

#tokenization
train2['simple_tokens'] = train2['text_hero'].map(simple_preprocess)
#output
y = train2['target']
#num feats
feats = get_feats(train2)

# Embeddings

In [ ]:

model_w2v = Word2Vec(train2['simple_tokens'], size=50, window=3, seed=0)

In [ ]:
print(model_w2v)

Word2Vec(vocab=2680, size=50, alpha=0.025)


In [ ]:
X_w2v_embedding = get_doc2vec_X(model_w2v, train2['simple_tokens'])
print(X_w2v_embedding)



/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning:

Mean of empty slice.

/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:170: RuntimeWarning:

invalid value encountered in double_scalars



[[ 0.10299411 -0.40191525  0.20049639 ...  0.32855585  0.19167137
   0.5453486 ]
 [ 0.10710851 -0.4194923   0.20801027 ...  0.34525248  0.20134237
   0.5757696 ]
 [ 0.04356027 -0.16717868  0.08565561 ...  0.14018156  0.08618589
   0.23345327]
 ...
 [ 0.09478268 -0.3711911   0.18791606 ...  0.30046025  0.17895092
   0.5035943 ]
 [ 0.07152266 -0.2928303   0.14313339 ...  0.23178186  0.13906536
   0.40198767]
 [ 0.12232207 -0.46423885  0.23901184 ...  0.38720113  0.22838014
   0.64138347]]


In [ ]:
X_w2v_embedding

In [ ]:
#word2vec
model_w2v = Word2Vec(train2['simple_tokens'], size=50, window=3, seed=0)
X_w2v_embedding = get_doc2vec_X(model_w2v, train2['simple_tokens'])
train2_w2v_encoding = pd.DataFrame(X_w2v_embedding)
X_w2v_embedding_feats = pd.merge(train2_w2v_encoding,train2[feats],left_index=True, right_index=True)

In [ ]:
#fasttext
model = FastText(train2['simple_tokens'], size=100, window=4, seed=0)

X_fasttext_embedding = get_doc2vec_X(model, train2['simple_tokens'])
train2_fasttext_encoding = pd.DataFrame(X_fasttext_embedding)
X_fasttext_embedding_feats = pd.merge(train2_fasttext_encoding,train2[feats],left_index=True, right_index=True)

## Models

In [ ]:
catboost = ctb.CatBoostClassifier(max_depth=8, n_estimators=100, verbose=0, random_state=0, custom_metric='AUC')
xgboost = xgb.XGBClassifier(max_depth=8, n_estimators=100, verbose=0, random_state=0, custom_metric='AUC')

def train_model(main_model, X_value, embedding_model, feats, tags):
    w2v_model_xgb = main_model
    w2v_model_xgb.fit(X_value, y)
    plot_learning_curve(w2v_model_xgb, X_value, y, cv=3, random_state=2019, shuffle=True)

    cvs = cross_val_score(w2v_model_xgb, X_value, y, scoring='accuracy', cv=3).mean()
    print(cvs)

    log_to_neptune(embedding_model, main_model, feats, tags, cvs)

word2vec

In [ ]:
#only text_hero, ctb
w2v_model_ctb = ctb.CatBoostClassifier(max_depth=8, n_estimators=100, verbose=0, random_state=0, custom_metric='AUC')
w2v_model_ctb.fit(X_w2v_embedding, y)
plot_learning_curve(w2v_model_ctb, X_w2v_embedding, y, cv=3, random_state=2019, shuffle=True)

In [ ]:
w2v_model_ctb.save_model('/content/drive/MyDrive/Konkursy Kaggle/Disaster Tweets/w2v_model_ctb_new')

In [ ]:
cvs = cross_val_score(w2v_model_ctb, X_w2v_embedding, y, scoring='accuracy', cv=3).mean()
print(cvs)

In [ ]:
log_to_neptune("word2vec", w2v_model_ctb, ['text_hero'], ['w2v', 'text_hero', 'ctb'], cvs)

In [ ]:
#text_hero+ feature engineering, ctb
w2v_model_ctb = ctb.CatBoostClassifier(max_depth=8, n_estimators=100, verbose=0, random_state=0, custom_metric='AUC')
w2v_model_ctb.fit(X_w2v_embedding_feats, y)
plot_learning_curve(w2v_model_ctb, X_w2v_embedding_feats, y, cv=3, random_state=2019, shuffle=True)

In [ ]:
cvs = cross_val_score(w2v_model_ctb, X_w2v_embedding_feats, y, scoring='accuracy', cv=3).mean()
print(cvs)

In [ ]:
log_to_neptune("word2vec", "ctb.CatBoostClassifier(max_depth=8, n_estimators=100, verbose=0, random_state=0, custom_metric='AUC')", ['text_hero'], ['w2v', 'feature_engineering', 'ctb'], cvs)

In [ ]:
#only text_hero, xgb
w2v_model_xgb = xgb.XGBClassifier(max_depth=8, n_estimators=100, verbose=0, random_state=0, custom_metric='AUC')
w2v_model_xgb.fit(X_w2v_embedding, y)
plot_learning_curve(w2v_model_xgb, X_w2v_embedding, y, cv=3, random_state=2019, shuffle=True)

cvs = cross_val_score(w2v_model_xgb, X_w2v_embedding, y, scoring='accuracy', cv=3).mean()
print(cvs)

log_to_neptune("word2vec", "xgb.XGBClassifier(max_depth=8, n_estimators=100, verbose=0, random_state=0, custom_metric='AUC')", ['text_hero'], ['w2v', 'text_hero', 'xgb'], cvs)

In [ ]:
#text_hero+ feats, xgb
w2v_model_xgb = xgb.XGBClassifier(max_depth=8, n_estimators=100, verbose=0, random_state=0, custom_metric='AUC')
w2v_model_xgb.fit(X_w2v_embedding_feats, y)
plot_learning_curve(w2v_model_xgb, X_w2v_embedding_feats, y, cv=3, random_state=2019, shuffle=True)

cvs = cross_val_score(w2v_model_xgb, X_w2v_embedding_feats, y, scoring='accuracy', cv=3).mean()
print(cvs)

log_to_neptune("word2vec", "xgb.XGBClassifier(max_depth=8, n_estimators=100, verbose=0, random_state=0, custom_metric='AUC')", feats, ['w2v', 'feature_engineering', 'xgb'], cvs)

In [ ]:
train_model(xgboost, X_w2v_embedding_feats, "word2vec", feats, ['w2v', 'feature_engineering', 'xgb'])

Fasttext

In [ ]:
#only text_hero, ctb
fasttext_model = ctb.CatBoostClassifier(max_depth=8, n_estimators=100, verbose=0, random_state=0, custom_metric='AUC')
fasttext_model.fit(X, y)
plot_learning_curve(fasttext_model, X, y, cv=3, random_state=2019, shuffle=True)

In [ ]:
"""
project["parameters"] = "max_depth:8, n_estimators:100, verbose:0, random_state:0, custom_metric:'AUC'"
project["feats"] = ["text_hero"]
project["sys/tags"].add(["fasttext", "text_hero"])
project["train/accuracy"] = 0.62
project.stop()
"""

In [ ]:
cross_val_score(fasttext_model, X, y, scoring='accuracy', cv=3).mean()

In [ ]:
#text_hero+numerical, ctb
fasttext_model2 = ctb.CatBoostClassifier(max_depth=8, n_estimators=100, verbose=0, random_state=0, custom_metric='AUC')
fasttext_model2.fit(X_all, y)
#plot_learning_curve(fasttext_model2, X_all, y, cv=3, random_state=2019, shuffle=True)

In [ ]:
cvs = cross_val_score(fasttext_model2, X_all, y, scoring='accuracy', cv=3).mean()

In [ ]:
#params = "max_depth=8, n_estimators=100, verbose=0, random_state=0, custom_metric='AUC'"
#log_to_neptune(fasttext_model2, params, feats, ['fasttext', 'text_hero', 'feature_engineering', 'ctb'], cvs)

In [ ]:
#only text_hero, xgb
fasttext_model_xgb = xgb.XGBClassifier(max_depth=8, n_estimators=100, verbose=0, random_state=0, custom_metric='AUC')
fasttext_model_xgb.fit(X, y)
plot_learning_curve(fasttext_model_xgb, X, y, cv=3, random_state=2019, shuffle=True)

In [ ]:
csv = cross_val_score(fasttext_model_xgb, X, y, scoring='accuracy', cv=3).mean()
print(csv)

In [ ]:
log_to_neptune(fasttext_model_xgb, params, ['text_hero'], ['fasttext', 'text_hero', 'xgb'], cvs)

In [ ]:
#text_hero+numerical, xgb
fasttext_model_xgb2 = xgb.XGBClassifier(max_depth=8, n_estimators=100, verbose=0, random_state=0, custom_metric='AUC')
fasttext_model_xgb2.fit(X_all, y)
plot_learning_curve(fasttext_model_xgb2, X_all, y, cv=3, random_state=2019, shuffle=True)

In [ ]:
csv = cross_val_score(fasttext_model_xgb2, X_all, y, scoring='accuracy', cv=3).mean()
print(csv)

In [ ]:
log_to_neptune(fasttext_model_xgb2, fasttext_model_xgb2, feats, ['fasttext', 'text_hero', 'feature_engineering', 'xgb'], csv)

# **Other**

In [ ]:
#prepare kaggle submission
test2['simple_tokens'] = test2['text_hero'].map(simple_preprocess)
X_test_encoded = get_doc2vec_X(model_w2v, test2['simple_tokens'])
test2_w2v2_encoding = pd.DataFrame(X_test_encoded)
X_test_all2 = pd.merge(test2_w2v2_encoding,test2[feats],left_index=True, right_index=True)
test2['target'] = w2v_model_xgb.predict(X_test_all2).astype(int)
test2[ ['id', 'target'] ].to_csv('/content/drive/MyDrive/Konkursy Kaggle/Disaster Tweets/xgb_w2v_allfeats.csv', index=False)

In [ ]:
# Create a Pickle file using serialization 
pickle_out = open("/content/drive/MyDrive/Konkursy Kaggle/Disaster Tweets/model_w2v_4.pkl","wb")
pickle.dump(model_w2v, pickle_out)
pickle_out.close()